**Tomato Leaf Disease Prediction**

https://github.com/krishnaik06/Tomato-Leaf-Disease-Prediction/blob/master/Transfer%20Learning%20Inception%20V3.ipynb

https://www.kaggle.com/datasets/kaustubhb999/tomatoleaf/code

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d noulam/tomato

 98% 729M/743M [00:08<00:00, 125MB/s]
100% 743M/743M [00:08<00:00, 93.9MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/tomato.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

from keras import Sequential
from keras.layers import Dense,MaxPooling2D,Conv2D,Flatten,BatchNormalization,Dropout,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from glob import glob

In [5]:
# Create CNN model

model = Sequential()
inputShape=(256,256,3)
chanDim=-1

model.add(Conv2D(128, (3, 3), padding="same", activation="relu", input_shape=inputShape))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(1024, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 128)     3584      
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 128)    512       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 85, 85, 128)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 85, 85, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 85, 85, 128)       147584    
                                                                 
 batch_normalization_1 (Batc  (None, 85, 85, 128)      5

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Data Agumentation

train_dataMaker = ImageDataGenerator(
    rescale=1./256,
    shear_range=0.2,
    zoom_range=0.3,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    rotation_range=40
)

validation_dataMaker = ImageDataGenerator(rescale = 1./256)

In [9]:
# Generator

train_ds = train_dataMaker.flow_from_directory(
    directory = '/content/New Plant Diseases Dataset(Augmented)/train',
    color_mode = 'rgb',
    interpolation = 'bicubic',
    batch_size = 32,
    target_size = (256,256),
    class_mode = 'categorical',
)

validation_ds = validation_dataMaker.flow_from_directory(
    directory = '/content/New Plant Diseases Dataset(Augmented)/valid',
    color_mode = 'rgb',
    interpolation = 'bicubic',
    batch_size = 32,
    target_size = (256,256),
    class_mode = 'categorical',
)

Found 18345 images belonging to 10 classes.
Found 4585 images belonging to 10 classes.


In [ ]:
history = model.fit(
    train_ds,
    epochs = 40,
    validation_data = validation_ds,
    steps_per_epoch=len(train_ds),
    validation_steps=len(validation_ds),
  )

Epoch 1/40
574/574 [==============================] - 471s 787ms/step - loss: 1.3077 - accuracy: 0.6001 - val_loss: 1.3180 - val_accuracy: 0.6266
Epoch 2/40
574/574 [==============================] - 443s 772ms/step - loss: 0.7299 - accuracy: 0.7597 - val_loss: 4.4568 - val_accuracy: 0.4031
Epoch 3/40
574/574 [==============================] - 438s 763ms/step - loss: 0.5805 - accuracy: 0.8086 - val_loss: 8.0306 - val_accuracy: 0.2567
Epoch 4/40
574/574 [==============================] - 442s 770ms/step - loss: 0.5076 - accuracy: 0.8299 - val_loss: 2.4384 - val_accuracy: 0.5710
Epoch 5/40
574/574 [==============================] - 440s 766ms/step - loss: 0.4500 - accuracy: 0.8463 - val_loss: 1.3115 - val_accuracy: 0.7152
Epoch 6/40
574/574 [==============================] - 441s 768ms/step - loss: 0.4181 - accuracy: 0.8593 - val_loss: 2.2242 - val_accuracy: 0.5834
Epoch 7/40
574/574 [==============================] - 437s 762ms/step - loss: 0.3786 - accuracy: 0.8703 - val_loss: 5.1468 -

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history['accuracy'], color = 'red', label = 'train')
plt.plot(history.history['val_accuracy'], color = 'green', label = 'validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], color = 'red', label = 'train')
plt.plot(history.history['val_loss'], color = 'green', label = 'validation')
plt.legend()
plt.show()

In [ ]:
import cv2

test_img1 = cv2.imread('/content/Early Blight Spot.jpg')
test_img2 = cv2.imread('/content/Leaf Mold.jpg')
test_img3 = cv2.imread('/content/Septoria Leaf Spot.jpg')
plt.imshow(test_img1)
plt.imshow(test_img2)
plt.imshow(test_img3)

In [ ]:
test_img1 = cv2.resize(test_img1, (256,256))
test_input = test_img1.reshape((1,256,256,3))

In [ ]:
model.predict(test_input)

In [ ]:
scores = model.evaluate(validation_ds)
print(f"Test Accuracy: {round(scores[1],4)*100}%")